In [18]:
from __future__ import print_function
from sklearn.cluster import KMeans
import numpy as np
import torch
import requests
from pathlib import Path
import pickle
import gzip
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch import optim

DATA_PATH=Path("data")
FilePath=DATA_PATH/"mnist"
URL = "http://deeplearning.net/data/mnist/"
FILENAME = "mnist.pkl.gz"

FilePath.mkdir(parents=True, exist_ok=True)

if not (FilePath/FILENAME).exists():
    content=requests.get(URL+FILENAME).content
    (FilePath/FILENAME).open("w+b").write(content)

with gzip.open((FilePath/FILENAME).as_posix(),"rb") as f:
    ((x_train, y_train), (x_valid, y_valid),_)=pickle.load(f, encoding="latin-1")
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape)   
(x_train, y_train, x_valid, y_valid)=map(torch.tensor, (x_train, y_train, x_valid, y_valid))



class Mnist_Logistic(nn.Module):
    def __init__(self):
        super(Mnist_Logistic, self).__init__()
        self.lin=nn.Linear(784,10)
        
        
    def forward(self, xb):
        return self.lin(xb)
    
loss_func=F.cross_entropy

def get_data(x_train, y_train, x_valid, y_valid):
    train_ds=TensorDataset(x_train, y_train)
    train_dl=DataLoader(train_ds, batch_size=bs)

    valid_ds=TensorDataset(x_valid, y_valid)
    valid_dl=DataLoader(valid_ds, batch_size=bs*2)
    return train_dl, valid_dl

def get_model(lr):
    model=Mnist_Logistic()
    opt=optim.SGD(model.parameters(), lr=lr)
    return model, opt

def fit(train_dl, valid_dl, model, loss_func, opt, epochs):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            pred=model(xb)
            loss=loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()
        model.eval()
        with torch.no_grad():
            valid_loss=sum(loss_func(model(xb), yb) for xb, yb in valid_dl)
        print(epoch, valid_loss/len(valid_dl))        

class Mnist_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2=nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv3=nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1)
    def forward(self, xb):
        xb=xb.view(-1,1,28,28)
        xb=F.relu(self.conv1(xb))
        xb=F.relu(self.conv2(xb))
        xb=F.relu(self.conv3(xb))
        xb=F.avg_pool2d(xb,4)
        return xb.view(-1, xb.size(1))
    
bs=64
lr=0.05
epochs=10

# model,opt=get_model(lr)
train_dl, valid_dl=get_data(x_train, y_train, x_valid, y_valid)
# fit(train_dl, valid_dl, model, loss_func, opt, epochs)        
model=Mnist_CNN()
# print(type(model.parameters()))
params = list(model.parameters())
# print(params[0].size()) 
for i in range(len(params)):
    print(i, params[i].size())  # conv1's .weight

opt=optim.SGD(model.parameters(), lr=lr, momentum=0.9)
fit(train_dl, valid_dl, model, loss_func, opt, epochs)        

(50000, 784) (50000,) (10000, 784) (10000,)
0 torch.Size([16, 1, 3, 3])
1 torch.Size([16])
2 torch.Size([16, 16, 3, 3])
3 torch.Size([16])
4 torch.Size([10, 16, 3, 3])
5 torch.Size([10])
0 tensor(0.5154)
1 tensor(0.3788)
2 tensor(0.3450)
3 tensor(0.3838)
4 tensor(0.3209)
5 tensor(0.2295)
6 tensor(0.2477)
7 tensor(0.1912)
8 tensor(0.1804)
9 tensor(0.1744)
